In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import timedelta

In [13]:
ticker = '7203.T'
stock = yf.Ticker(ticker)
info = stock.info
        # financials = stock.financials
        # balance_sheet = stock.balance_sheet
# cashflow = stock.cashflow
# earnings = stock.get_earnings
balance = stock.balance_sheet
cashflow = stock.cashflow
income = stock.financials
info = stock.info
debt_equity = info.get('debtToEquity', np.nan)

In [21]:
def get_financials(ticker):
    try:
        stock = yf.Ticker(ticker)
        # balance = stock.balance_sheet # 4年分の貸借対照表を取得
        # cashflow = stock.cashflow # 4年分のキャッシュフロー計算書を取得
        # income = stock.financials # 4年分の損益計算書
        info = stock.info # infoは辞書型

        # メトリクスの取得
        # DEレシオ = 有利子負債 ÷ 自己資本
        debt_equity = info.get('debtToEquity', np.nan)
        # 流動比率 = 流動資産 ÷ 流動負債
        current_ratio = info.get('currentRatio', np.nan)
        # 株価純資産倍率（PBR） = 株価 ÷ 1株当たり純資産（BPS）
        price_book = info.get('priceToBook', np.nan)
        # ROE(%)
        roe = info.get('returnOnEquity', np.nan) * 100
        # ROA(%)
        roa = info.get('returnOnAssets', np.nan) * 100
        # 四半期利益成長率
        quarter_growth = info.get('earningsQuarterlyGrowth', np.nan)

        return {
            'Ticker': ticker,
            'Debt_Equity': debt_equity,
            'Current_Ratio': current_ratio,
            'Price_Book':price_book,
            'ROE(%)':roe,
            'ROA(%)':roa,
            'Quarter Growth': quarter_growth,
        }
    except Exception as e:
        print(f'Error fetching data for {ticker}: {e}')
        return None

In [27]:
tickers = ['7203.T','9432.T']

# 財務データの取得
financial_data = []
for ticker in tickers:
    data = get_financials(ticker)
    if data:
        financial_data.append(data)

# データフレームに変換する
df = pd.DataFrame(financial_data)

# スクリーニングの各条件を満たす場合にポイントを加算していく
df['Point'] = 0
df.loc[df['Debt_Equity'] < 0.5, 'Point'] += 10
df.loc[df['Current_Ratio'] > 1.5, 'Point'] += 10
df.loc[df['Price_Book'] < 1.5, 'Point'] += 10
df.loc[df['ROE(%)'] > 8, 'Point'] += 10
df.loc[df['ROA(%)'] > 6, 'Point'] += 10
df.loc[df['Quarter Growth'] > 0.10, 'Point'] += 10

# 結果の並び替え
df.sort_values('Point', ascending=False)

,Ticker,Debt_Equity,Current_Ratio,Price_Book,ROE(%),ROA(%),Quarter Growth,Point
0,7203.T,103.922,1.266,1.032106,11.652000,3.095,-0.369,20
1,9432.T,137.940,0.893,1.434013,10.086001,3.607,-0.053,20
